In [4]:
%%time
from pyspark import SparkContext,SparkConf
from pyspark.sql import HiveContext
from pyspark.sql.types import Row

hiveCntx = HiveContext(sc)
hiveCntx.setConf("hive.exec.dynamic.partition", "true") 
hiveCntx.setConf("hive.exec.dynamic.partition.mode", "nonstrict")
BRO_CONN_RAW_DF =  hiveCntx.sql('from shprd_sda_infra_enc.bro_conn_logs_raw_test select *')
BRO_CONN_RAW_RDD = BRO_CONN_RAW_DF.rdd
SRC_IP_CNT = BRO_CONN_RAW_RDD.map(lambda inRec:((inRec[2],inRec[24],inRec[25],inRec[26]),1)).reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[1],False).map(lambda record:Row(src_ip = record[0][0], ip_count=record[1], log_year=record[0][1], log_month=record[0][2], log_day=record[0][3])).toDF()
  #SRC_IP_CNT_DESC = SRC_IP_CNT.takeOrdered(200,key=lambda record: -record[1]).parallelize().map(lambda record:Row(src_ip = record[0], ip_count=record[4], log_year=record[1],log_month=record[2],log_day=record[3]))
SRC_IP_CNT.write.mode('append').format('orc').partitionBy('log_year','log_month','log_day').saveAsTable("moneyball.der_bro_conn_src_ip")
  
